# Übung: MIP für makespan


Die Firma *Print4U* betreibt eine Druckerei mit verschiedenen Druckern. Ziel der Firma ist es die Druckaufträge $J$ so auf die Drucker $M$ zu verteilen, sodass alle Aufträge so früh wie möglich abgearbeitet sind. *Print4U* beauftragt Sie eine optimale Aufteilung zu finden. Sie haben bereits erkannt, dass das aktuelle Problem eine Instanz des *minimum makespan*-Problems ist (siehe Abschnitt 3.6) und Sie wollen das MIP aus dem Skript benutzen. 

$$
\begin{align*}
\min T & \\
\text{unter } 
\sum_{a\in J} x_{ap}d_{ap} \leq T&\text{ für alle }p\in M\\
\sum_{p\in M} x_{ap} = 1 & \text{ für alle }a\in J\\
x_{ap}\in\{0,1\} &\text{ für alle } a\in J\text{ und }p\in M
\end{align*}
$$


Sie stellen nun fest, dass es bei *Print4U* weitere Bedingungen gibt:
1. Ein Teil der Druckaufträge soll in Farbe gedruckt werden. Fur jeden Auftrag $a \in J$ gibt
es einen Eintrag $f_a\in\{0,1\}$, der angibt, ob das Dokument in Farbe gedruckt werden soll. Dabei
heißt $f_a = 1$, dass das Dokument in Farbe gedruckt werden muss, und $f_a = 0$, dass keine
Farbe notwenig ist.
Leider können nicht alle Drucker farbig drucken. Für jeden Drucker $p \in M$ gibt es einen
Eintrag $c_p\in\{0,1\}$ mit $c_p = 1$, wenn der Drucker farbig drucken kann (ansonsten $c_p = 0$). Farbdrucker können naturlich trotzdem schwarz-weiß Dokumente drucken. 
2. Alle Drucker können durch ein Zusatzmodul erweitert werden, sodass die Dokumente automatisch gebunden werden können. Das Modul nimmt allerdings viel Platz in Anspruch.
Dadurch können Drucker mit Modul nur noch 10 Druckaufträge annehmen.
Fur jeden Druckauftrag $a \in J$ gibt es wieder einen Eintrag $b_a\in\{0,1\}$ mit $b_a = 1$, falls der Auftrag
gebunden werden muss. Drucker mit Modul können auch Druckaufträge annehmen, die
nicht gebunden werden mussen. Diese Aufträge werden allerdings trotzdem in der Grenze
von 10 Aufträgen beachtet.
3. Alle Drucker haben die Möglichkeit zu dem Einzug von Din A4-Blättern einen zweiten
Papiereinzug anzusteuern. Der zweite Papiereinzug kann entweder mit Din A3 oder mit
Din A5 Blättern bestückt werden. Da das Papier aber bereits am Morgen für den ganzen
Tag bestückt wird, kann ein Drucker nicht zwischen A3- und A5-Blättern wechseln. A4-Blätter sind aus dem ersten Einzug immer verfügbar. 
Es gibt wieder fur jeden Druckauftrag $a \in J$ Einträge $k_a\in\{0,1\}$ und $g_a\in\{0,1\}$ mit $k_a = 1$, falls $a$ ein
 A5-Dokument ist, $g_a = 1$, falls $a$ ein A3-Dokument ist und $k_a = g_a = 0$,
falls $a$ ein A4-Dokument ist.

Erweitern Sie jeweils das MIP um weitere Variablen und Bedingungen, sodass die Zusatzbedingungen eingehalten werden. Lösen Sie dafür das Problem zunächst in Textform und erweitern
anschließend den Programmcode für *minimum makespan*. Geben Sie sowohl ihre schriftliche
Lösung als auch das modifizierte Jupyter Notebook ab.

## Das MIP

Wir starten mit zwei nötigen imports.

In [ ]:
# Installation des Pakets mip
# Wenn Sie _nicht_ in Google Colab arbeiten, müssen Sie eventuell manuell das Paket installieren 
# In dem Fall kommentieren Sie die nächste Zeile aus
!pip install mip==1.8.1

import mip
import random  ## Standard-Bibliothek zur Erzeugung von Zufallszahlen

Als erstes definieren wir eine Methode zur Erzeugung von Zufallsinstanzen. Die Funktion <code>random.randint(a,b)</code> liefert eine zufällig ganze Zahl zwischen a und b (einschließlich). Die Funktion <code>random.random()</code> liefert eine zufällige Zahl in dem Interval $[0,1]$.

In [ ]:
def random_instance(number_machines,number_jobs):
    random.seed(42)
    d = [[random.randint(1,20) for _ in range(number_machines)] for _ in range(number_jobs)]
    # Farbdruck
    f = [random.randint(0,1)  for _ in range(number_jobs)]   
    c = [random.randint(0,1) for _ in range(number_machines)]
    # Binden
    b = [1*(random.random()<0.05)  for _ in range(number_jobs)]
    # Blattgröße
    random_vec = [random.random()  for _ in range(number_jobs)]
    # Falls Zufallszahl klein, kleines Papier. Falls groß, großes Papier
    k = ([1*(a<0.2) for a in random_vec]) 
    g = ([1*(a>0.9) for a in random_vec])
    return d,f,c,b,k,g

number_jobs=300
number_machines=10
d,f,c,b,k,g=random_instance(number_machines,number_jobs)

Wir stellen das mip auf. Dazu definieren wir zunächst das mip-Objekt und fügen gleich Variablen hinzu, einmal die Variablen $x_{ap}\in\{0,1\}$ für Auftrag $a$ und Maschine $p$ und dann noch die makespan-Variable $T$, die alle (nicht-negativen) Werte annehmen darf. Durch Angabe von <code>var_type=mip.BINARY</code> definieren wir Binärvariablen. Mit <code>var_type=INTEGER</code> würden wir allgemeine Ganzzahlvariablen definieren.

In [ ]:
m=mip.Model()
x=[[m.add_var(var_type=mip.BINARY) for _ in range(number_machines)] for _ in range(number_jobs)]
T=m.add_var()

Wir fügen die Bedingung

$\sum_{a\in J}x_{ap}d_{ap}\leq T\quad\text{ für alle }p\in M$

hinzu.

In [ ]:
for p in range(number_machines):
    m+=mip.xsum(x[a][p]*d[a][p] for a in range(number_jobs))<=T

Schließlich müssen wir garantieren, dass auch jeder Auftrag bearbeitet wird:

$\sum_{p\in M}x_{ap}=1\quad\text{für alle }a\in J$

In [ ]:
for a in range(number_jobs):
    m+=mip.xsum(x[a][p] for p in range(number_machines))==1

### Aufgabe: Farbdruck
Modellieren Sie die Farbdruckbedingungen (Punkt 1. oben) und erweitern Sie entsprechend das MIP.




In [ ]:
### Ihr Code hier ###


### Aufgabe: Bindung
Modellieren Sie die Bedingungen für die Papierbindung (Punkt 2. oben) und erweitern Sie entsprechend das MIP.

In [ ]:
### Ihr Code hier ###


### Aufgabe: Papierformate
Modellieren Sie die Bedingungen für die unterschiedlichen Papierformate (Punkt 3. oben) und erweitern Sie entsprechend das MIP.

In [ ]:
### Ihr Code hier ###


Wir starten die Optimierung:

In [ ]:
m.objective=mip.minimize(T)
m.verbose=False  # unterdrücke ausführliche Statusmeldungen
m.optimize()    

Schließlich ist's zweckmäßig sich die Lösung auszugeben:

In [ ]:
detailierte_ausgabe = False
if detailierte_ausgabe:
    for p in range(number_machines):
        print("Maschine {}:".format(p))
        jobs=[a for a in range(number_jobs) if x[a][p].x==1]
        print("  Aufträge: {}".format(jobs))
        job_lengths=[d[a][p] for a in jobs]
        print("  Auftragsdauern: {}".format(job_lengths))
        print("  Arbeitsdauer Maschine {}: {}".format(p,sum(job_lengths)))
    print("-------")
print("makespan: {}".format(T.x)) # Hier können numerische Ungenauigkeiten auftreten!